In [1]:
import binascii
import serial
import os
import cv2
from  matplotlib import pyplot as plt

In [2]:
def run_action(cmd):
    ser = serial.Serial("/dev/ttyPS0", 9600, timeout=5)
    cnt_err = 0
    while 1:
        test_read = ser.read()
        # print('test_read', test_read)
        cnt_err += 1
        if test_read== b'\xa3' or cnt_err == 50:
            break
    
    if cnt_err == 50:
        print('can not get REQ')
    else:
        print('read REQ finished!')
        ser.write(cmd2data(cmd))
        print('send action ok!')
    ser.close()
    
def crc_calculate(package):
    crc = 0
    for hex_data in package:

        b2 = hex_data.to_bytes(1, byteorder='little')
        crc = binascii.crc_hqx(b2, crc)

    return [(crc >> 8), (crc & 255)]    # 校验位两位

def cmd2data(cmd):
    cnt=0
    cmd_list=[]
    for i in cmd:
        
        cnt+=1
        cmd_list+=[ord(i)]
    cmd_list=[0xff,0xff]+[(cnt+5)>>8,(cnt+5)&255]+[0x01,(cnt+1)&255,0x03]+cmd_list
    cmd_list=cmd_list+crc_calculate(cmd_list)
    return cmd_list

def wait_req():
    ser = serial.Serial("/dev/ttyPS0", 9600, timeout=5)
    while 1:
        test_read=ser.read()
        if test_read== b'\xa3' :
            print('read REQ finished!') 
            break

In [3]:
def getpos():
    os.system('fswebcam  --no-banner --save temp.jpg -d /dev/video0 2> /dev/null')
    image = cv2.imread('temp.jpg')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret,thres = cv2.threshold(gray, 60, 255, 0)
    edges = cv2.Canny(thres, 50, 150)
    minLineLength = 130
    maxLineGap = 20
    lines = cv2.HoughLinesP(edges, rho=1, theta=3.1415926/180, threshold=40, minLineLength=minLineLength, maxLineGap=maxLineGap)
    for line in lines:
        print(line)
        for x1, y1, x2, y2 in line:
            k = (y2 - y1) / (x2 - x1)
            if k > 2:
                continue
            b = y1 - k * x1
            print(k, b)
            cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
    # plt.imshow(thres)
#     plt.imshow(image)
#     plt.show()
    return (b, k)
b_thres = 10
s_thres = 0.05

In [4]:
os.system('sh ./stop_sys_ttyPS0.sh')

32512

In [5]:
flag1 = False
flag2 = False

while not (flag1 and flag2):
    (b, s) = getpos()
    flag1 = False
    flag2 = False
    
    if b < -b_thres:
        wait_req()
        run_action('turn003L')
    elif b > b_thres:
        wait_req()
        run_action('turn003R')
    else:
        flag1 = True
        
    if s < -s_thres:
        wait_req()
        run_action('Left02move')
    elif s > s_thres:
        wait_req()
        run_action('Right02move')
    else:
        flag2 = True

(480, 640, 3)
4 0.5279503105590062
read REQ finished!


SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

In [ ]:
wait_req()
run_action('turn001L')

In [10]:
run_action('Forwalk05')

read REQ finished!
send action ok!


In [ ]:
def f1L():
    run_action('turn005L')
    wait_req()
    run_action('turn005L')
    wait_req()
    run_action('turn005L')
    wait_req()
    run_action('turn001L')
    wait_req()
    run_action('turn001L')
    wait_req()
    run_action('turn001L')
    wait_req()
    run_action('Stand')

KeyboardInterrupt: 